<a href="https://colab.research.google.com/github/thayeylolu/cyberbullying/blob/main/notebooks/cleaning_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
! pip install demoji

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Import Libaries

In [18]:
# importing libraries

import demoji
import json
import pandas as pd
import re, nltk
import warnings

from nltk.tokenize import word_tokenize
from nltk import word_tokenize
from nltk.corpus import stopwords

nltk.download('stopwords')
nltk.download('punkt')
stop_word = stopwords.words('english')
warnings.filterwarnings(action="ignore")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Mount Drive

In [19]:
from google.colab import drive
drive.mount('/drive')

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


## Read CSV

In [20]:
url = "/drive/MyDrive/NLP/cyberbullying_tweets.csv"
df = pd.read_csv(url)
df.head()

,tweet_text,cyberbullying_type
0,"In other words #katandandre, your food was cra...",not_cyberbullying
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying


## Checkin Unique Classes

In [21]:
df['cyberbullying_type'].unique()

array(['not_cyberbullying', 'gender', 'religion', 'other_cyberbullying',
       'age', 'ethnicity'], dtype=object)

## data cleaning steps

- do eda on all the datzset 
- replace emojis with word
- keep hastags, but remove # 
- remove usernames 
- remove punctuation
- remove stop words and two letter words
- tokenize


- find profanity words using word2vec or not <>
- add more words to the list of profanity???
- remove non english words
- contraction 


## List of possibel contractions in english
- Sourced from  https://gist.github.com/MLWhiz/a603656c482ce13f3f2affc1d35f287d

In [22]:
# refernce : https://gist.github.com/MLWhiz/a603656c482ce13f3f2affc1d35f287d

contraction_dict = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have"}


## Replace contraction with  the full word

Example: 

```python
replace_contractions("this's a text with contraction")
>>> 'this is a text with contraction'
```


In [23]:
def _get_contractions(contraction_dict):
    contraction_re = re.compile('(%s)' % '|'.join(contraction_dict.keys()))
    return contraction_dict, contraction_re
contractions, contractions_re = _get_contractions(contraction_dict)

def replace_contractions(text):
    def replace(match):
        return contractions[match.group(0)]
    return contractions_re.sub(replace, text)
    

### Cleaning Function
- Load Profanity Data sourced from  https://github.com/zacanger/profane-words/blob/master/words.json

In [24]:
def super_clean(df):
    profanity_url = "/drive/MyDrive/NLP/profanity_list.json"

    # Opening JSON file
    f = open(profanity_url) 
    profanity_data = json.load(f)

    # extract emojis
    def extract_emoji(txt):
        emoji_txt = demoji.findall(txt)
        emoji_keys = emoji_txt.keys()
        emoji_values = emoji_txt.values()
        return  ' '.join(list(map(str, emoji_keys))), ' '.join(list(map(str, emoji_values)))

    # extract hashtags
    def hashtags(txt):
        txt = re.findall("#([a-zA-Z0-9_]{1,50})", txt)
        return ' '.join(list(map(str, txt)))
    
    # extract mentions
    def mentions(txt):
        txt = re.findall("@([a-zA-Z0-9_]{1,50})", txt)
        return ' '.join(list(map(str, txt)))

    # lookup profanity manually
    def find_profanity(text):
        profanity_set = set() 
        for word in text.split(): 
            if word in profanity_data:
                profanity_set.add(word)
        return ' '.join(list(map(str, profanity_set)))

      
    df['profanity_list'] = df['tweet_text'].apply(lambda x: find_profanity(x)) # get curse words
    df['hashtags'] = df['tweet_text'].apply(lambda x: hashtags(x)) # get hashtags
    df['mentions'] = df['tweet_text'].apply(lambda x: mentions(x)) # get mentions
    df['emoji_names'] =  df['tweet_text'].apply(lambda x:extract_emoji(x)[1]) # get names of emojis

    
    df['clean_txt'] = df['tweet_text'].apply(lambda x: replace_contractions(x)) # replace contraction
    df['clean_txt'] = df['clean_txt'].apply(lambda x: ' '.join(word for word in x.split() if word not in stop_word or len(word) > 2)) # remove stopwords
    df['clean_txt'] =  df['clean_txt'].apply(lambda x: re.sub("@[A-Za-z0-9_]+","", x)) # mentions
    df['clean_txt'] =  df['clean_txt'].apply(lambda x: re.sub("#[A-Za-z0-9_]+","", x)) # remove hashtags

    df['clean_txt_emoji'] =   df['clean_txt'] + " " + df['emoji_names'] # add emoji as text to tweet
    df['clean_txt_emoji'] =  df['clean_txt_emoji'].str.lower() # to lower case
    df['clean_txt_emoji'] = df['clean_txt_emoji'].apply(lambda x: re.sub(r"http\S+", "", x)) # remove links
    df['clean_txt_emoji'] =   df['clean_txt_emoji']+ " " + df['hashtags'] # add hashtags as text to tweet
    df['clean_txt_emoji'] = df['clean_txt_emoji'].str.replace("[^a-zA-Z#]", " ") # punctuation
    df['clean_txt_emoji'] = df['clean_txt_emoji'].apply(lambda x: ' '.join(word for word in x.split() if len(word) > 2)) #remove stop words AGAIN
   
    df['clean_txt_emoji'] = df['clean_txt_emoji'].str.strip() # rmeove white space
    df['tokenized'] = df['clean_txt_emoji'].apply(word_tokenize) # tokenize
   
    return df


## Run cleaning Function

In [27]:
# cleaning_time < 5 minutes
clean_df = super_clean(df)


Save `clean_txt_emoji` and `cyberbullying_type` in a new dataframe called `train_df`

In [28]:
train_df = clean_df[['clean_txt_emoji', 'cyberbullying_type']]

### After cleaning 


In [29]:
clean_df.sample(4)

,tweet_text,cyberbullying_type,profanity_list,hashtags,mentions,emoji_names,clean_txt,clean_txt_emoji,tokenized
47281,“@tedddydgaf: I hate niggers .” You sound dumb...,ethnicity,niggers,,tedddydgaf,,“: I hate niggers .” You sound dumb fuck.,hate niggers you sound dumb fuck,"[hate, niggers, you, sound, dumb, fuck]"
28383,RT @gater_ebooks: Just trying to make people I...,other_cyberbullying,,,gater_ebooks,,RT : Just trying make people I care about gend...,just trying make people care about gender fluid,"[just, trying, make, people, care, about, gend..."
7500,Bullying e otimo!... Apartir do momento que nã...,not_cyberbullying,,,,,Bullying e otimo!... Apartir momento que não e...,bullying otimo apartir momento que praticado c...,"[bullying, otimo, apartir, momento, que, prati..."
28303,RT @DearIsamar_: *inserts boring ass halo eye ...,other_cyberbullying,ass,,DearIsamar_,,RT : *inserts boring ass halo eye look the TL ...,inserts boring ass halo eye look the,"[inserts, boring, ass, halo, eye, look, the]"


### Save cleaned dataset and train dataset

In [30]:

clean_df.to_csv("clean_df.csv")


In [31]:
train_df.to_csv("train_data.csv")

In [32]:
train_df.sample(10)

,clean_txt_emoji,cyberbullying_type
8305,dave chapelle offensive you not like rape anti...,gender
41241,sooo this fuck nigger still leaves the house w...,ethnicity
47170,dumb nigger just repeated what said fuck you a...,ethnicity
7309,bullying victims fight back with help from bra...,not_cyberbullying
21505,idiot you who play muslim muslim game since yo...,religion
32566,the exact same writing every girl who bullied ...,age
10200,twinkle twinkle little bitch close your legs s...,gender
24674,that face,other_cyberbullying
3602,envious your education knowing how interpret s...,not_cyberbullying
19247,such death destruction always accompanies jiha...,religion
